In [1]:
import os
import pandas as pd

# 1. Import Dataset

In [2]:
DATA_FOLDER_NAME = 'data'

sampleOrigFiles = str(os.getcwd())+"/" + DATA_FOLDER_NAME + "\sample_orig_*.txt"
outputOrigFile = 'SampleOriginationCombined.csv'

orig_headers = ['CREDIT_SCORE','FIRST_PAYMENT_DATE','FIRST_TIME_HOMEBUYER_FLAG','MATURITY_DATE','MSA','MI_PCT',
                'NUMBER_OF_UNITS','OCCUPANCY_STATUS','CLTV','DTI','ORIGINAL_UPB','LTV','ORIGINAL_INTEREST_RATE',
                'CHANNEL','PPM','AMORTIZATION_TYPE','PROPERTY_STATE', 'PROPERTY_TYPE','POSTAL_CODE',
                'LOAN_SEQUENCE_NUMBER','LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM','NUMBER_OF_BORROWERS','SELLER_NAME',
                'SERVICER_NAME','SUPER_CONFORMING_FLAG','PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 
                'PROGRAM_INDICATOR', 'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION_METHOD', 'IO_INDICATOR']

orig_types = {'CREDIT_SCORE':float, 'FIRST_PAYMENT_DATE':str, 'FIRST_TIME_HOMEBUYER_FLAG':str, 'MATURITY_DATE':str,
              'MSA':str, 'MI_PCT':float, 'NUMBER_OF_UNITS':float, 'OCCUPANCY_STATUS':str, 'CLTV':float, 'DTI':float,
              'ORIGINAL_UPB':float, 'LTV':float, 'ORIGINAL_INTEREST_RATE':float, 'CHANNEL':str, 'PPM': str,
              'AMORTIZATION_TYPE':str ,'PROPERTY_STATE':str, 'PROPERTY_TYPE':str ,'POSTAL_CODE':str,
              'LOAN_SEQUENCE_NUMBER':str, 'LOAN_PURPOSE':str, 'ORIGINAL_LOAN_TERM':float, 'NUMBER_OF_BORROWERS':float,
              'SELLER_NAME': str, 'SERVICER_NAME':str, 'SUPER_CONFORMING_FLAG':str,
              'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER':str, 'PROGRAM_INDICATOR':str, 'RELIEF_REFINANCE_INDICATOR':str,
              'PROPERTY_VALUATION_METHOD':str, 'IO_INDICATOR':str}

orig_dates = ['FIRST_PAYMENT_DATE', 'MATURITY_DATE']

samplePerfFiles = str(os.getcwd())+"/" + DATA_FOLDER_NAME + "\sample_svcg_*.txt"
outputPerfFile = 'SamplePerformanceCombined.csv'

perf_headers = ['LOAN_SEQUENCE_NUMBER','MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB',
                'CURRENT_LOAN_DELINQUENCY_STATUS','LOAN_AGE','REMAINING_MONTHS_TO_LEGAL_MATURITY', 
                'DEFECT_SETTLEMENT_DATE','MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 
                'ZERO_BALANCE_EFFECTIVE_DATE','CURRENT_INTEREST_RATE','CURRENT_DEFERRED_UPB','DDLPI',
                'MI_RECOVERIES', 'NET_SALE_PROCEEDS','NON_MI_RECOVERIES','EXPENSES', 'LEGAL_COSTS',
                'MAINTENANCE_AND_PRESERVATION_COSTS','TAXES_AND_INSURANCE','MISCELLANEOUS_EXPENSES',
                'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST','STEP_MODIFICATION_FLAG','DEFERRED_PAYMENT_PLAN',
                'ELTV','ZERO_BALANCE_REMOVAL_UPB','DELINQUENT_ACCRUED_INTEREST','DELINQUENCY_DUE_TO_DISASTER',
                'BORROWER_ASSISTANCE_STATUS_CODE','CURRENT_MONTH_MODIFICATION_COST','INTEREST_BEARING_UPB']

perf_types = {'LOAN_SEQUENCE_NUMBER':str, 'MONTHLY_REPORTING_PERIOD':str, 'CURRENT_ACTUAL_UPB':float,
              'CURRENT_LOAN_DELINQUENCY_STATUS':str, 'LOAN_AGE':float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY':float, 
              'DEFECT_SETTLEMENT_DATE':str, 'MODIFICATION_FLAG':str, 'ZERO_BALANCE_CODE':str,
              'ZERO_BALANCE_EFFECTIVE_DATE':str, 'CURRENT_INTEREST_RATE':float, 'CURRENT_DEFERRED_UPB':float,
              'DDLPI': str,'MI_RECOVERIES':float, 'NET_SALE_PROCEEDS':str ,'NON_MI_RECOVERIES':float,'EXPENSES':float,
              'LEGAL_COSTS':float ,'MAINTENANCE_AND_PRESERVATION_COSTS':float, 'TAXES_AND_INSURANCE':float,
              'MISCELLANEOUS_EXPENSES':float,'ACTUAL_LOSS_CALCULATION':float, 'MODIFICATION_COST':float,
              'STEP_MODIFICATION_FLAG':str, 'DEFERRED_PAYMENT_PLAN':str, 'ELTV':float, 'ZERO_BALANCE_REMOVAL_UPB':float,
              'DELINQUENT_ACCRUED_INTEREST':float, 'DELINQUENCY_DUE_TO_DISASTER':str,'BORROWER_ASSISTANCE_STATUS_CODE':str,
              'CURRENT_MONTH_MODIFICATION_COST':float, 'INTEREST_BEARING_UPB':float}

perf_dates = ['DEFECT_SETTLEMENT_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE', 'DDLPI']

In [3]:
from FREDDIEMAC_preprocessing import createOriginationCombined, createPerformanceCombined

#createOriginationCombined(sampleOrigFiles, orig_headers, orig_types, outputOrigFile)
#createPerformanceCombined(samplePerfFiles, perf_headers, perf_types, outputPerfFile)

In [4]:
df_orig = pd.read_csv(outputOrigFile, dtype=orig_types)

In [5]:
df_perf = pd.read_csv(outputPerfFile, dtype=perf_types)

In [6]:
df_orig

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,MSA,MI_PCT,NUMBER_OF_UNITS,OCCUPANCY_STATUS,CLTV,DTI,...,SELLER_NAME,SERVICER_NAME,SUPER_CONFORMING_FLAG,PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER,PROGRAM_INDICATOR,RELIEF_REFINANCE_INDICATOR,PROPERTY_VALUATION_METHOD,IO_INDICATOR,QUARTER,YEAR
0,686.0,201604,N,203603,39820,0.0,1.0,P,65.0,41.0,...,Other sellers,Other servicers,N,X,9,X,9,N,1,16
1,774.0,201603,Y,204602,29620,30.0,1.0,P,95.0,30.0,...,Other sellers,Other servicers,N,X,9,X,9,N,1,16
2,808.0,201604,N,203103,30700,0.0,1.0,P,81.0,15.0,...,Other sellers,Other servicers,N,X,9,X,9,N,1,16
3,784.0,201603,N,204602,0,0.0,1.0,P,82.0,42.0,...,Other sellers,Other servicers,N,X,9,X,9,N,1,16
4,798.0,201603,N,204602,47664,0.0,1.0,P,28.0,22.0,...,Other sellers,Other servicers,N,X,9,X,9,N,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,733.0,201801,N,204712,35614,0.0,1.0,P,80.0,39.0,...,Other sellers,Other servicers,N,X,9,X,2,N,4,17
99996,799.0,201801,N,204712,19740,0.0,1.0,P,60.0,44.0,...,Other sellers,Other servicers,N,X,9,X,2,N,4,17
99997,812.0,201801,N,204712,19740,0.0,1.0,P,80.0,23.0,...,Other sellers,Other servicers,N,X,9,X,2,N,4,17
99998,806.0,201712,N,203711,0,0.0,1.0,S,80.0,20.0,...,Other sellers,Other servicers,N,X,9,X,2,N,4,17


In [7]:
df_perf

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,LOAN_AGE,REMAINING_MONTHS_TO_LEGAL_MATURITY,DEFECT_SETTLEMENT_DATE,MODIFICATION_FLAG,ZERO_BALANCE_CODE,ZERO_BALANCE_EFFECTIVE_DATE,...,ELTV,ZERO_BALANCE_REMOVAL_UPB,DELINQUENT_ACCRUED_INTEREST,DELINQUENCY_DUE_TO_DISASTER,BORROWER_ASSISTANCE_STATUS_CODE,CURRENT_MONTH_MODIFICATION_COST,INTEREST_BEARING_UPB,QUARTER,YEAR,REO_ACQUISITION
0,0000006,201602,125000.00,0,0.0,241.0,X,N,0,189901,...,NaN,NaN,NaN,NaN,NaN,NaN,125000.00,1,16,0
1,0000006,201603,125000.00,0,1.0,240.0,X,N,0,189901,...,NaN,NaN,NaN,NaN,NaN,NaN,125000.00,1,16,0
2,0000006,201604,124000.00,0,2.0,239.0,X,N,0,189901,...,NaN,NaN,NaN,NaN,NaN,NaN,124000.00,1,16,0
3,0000006,201605,124000.00,0,3.0,238.0,X,N,0,189901,...,NaN,NaN,NaN,NaN,NaN,NaN,124000.00,1,16,0
4,0000006,201606,124000.00,0,4.0,237.0,X,N,0,189901,...,NaN,NaN,NaN,NaN,NaN,NaN,124000.00,1,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4597177,0360884,202108,233627.12,0,45.0,195.0,X,N,0,189901,...,38.0,NaN,NaN,NaN,NaN,NaN,233627.12,4,17,0
4597178,0360884,202109,232571.30,0,46.0,194.0,X,N,0,189901,...,38.0,NaN,NaN,NaN,NaN,NaN,232571.30,4,17,0
4597179,0360884,202110,231512.07,0,47.0,193.0,X,N,0,189901,...,40.0,NaN,NaN,NaN,NaN,NaN,231512.07,4,17,0
4597180,0360884,202111,230449.42,0,48.0,192.0,X,N,0,189901,...,40.0,NaN,NaN,NaN,NaN,NaN,230449.42,4,17,0


### 1.1 Data Exploration

In [8]:
df_perf['LOAN_SEQUENCE_NUMBER'].nunique()

90167

In [9]:
df_perf.loc[df_perf['LOAN_SEQUENCE_NUMBER'] == "0360884"]

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,LOAN_AGE,REMAINING_MONTHS_TO_LEGAL_MATURITY,DEFECT_SETTLEMENT_DATE,MODIFICATION_FLAG,ZERO_BALANCE_CODE,ZERO_BALANCE_EFFECTIVE_DATE,...,ELTV,ZERO_BALANCE_REMOVAL_UPB,DELINQUENT_ACCRUED_INTEREST,DELINQUENCY_DUE_TO_DISASTER,BORROWER_ASSISTANCE_STATUS_CODE,CURRENT_MONTH_MODIFICATION_COST,INTEREST_BEARING_UPB,QUARTER,YEAR,REO_ACQUISITION
4597156,0360884,201911,254044.17,0,24.0,216.0,X,N,0,189901,...,999.0,NaN,NaN,NaN,NaN,NaN,254044.17,4,17,0
4597157,0360884,201912,253054.28,0,25.0,215.0,X,N,0,189901,...,55.0,NaN,NaN,NaN,NaN,NaN,253054.28,4,17,0
4597158,0360884,202001,253054.28,0,26.0,214.0,X,N,0,189901,...,55.0,NaN,NaN,NaN,NaN,NaN,253054.28,4,17,0
4597159,0360884,202002,251064.90,0,27.0,213.0,X,N,0,189901,...,53.0,NaN,NaN,NaN,NaN,NaN,251064.90,4,17,0
4597160,0360884,202003,251064.90,0,28.0,212.0,X,N,0,189901,...,52.0,NaN,NaN,NaN,NaN,NaN,251064.90,4,17,0
4597161,0360884,202004,250065.39,0,29.0,211.0,X,N,0,189901,...,52.0,NaN,NaN,NaN,NaN,NaN,250065.39,4,17,0
4597162,0360884,202005,249062.65,0,30.0,210.0,X,N,0,189901,...,51.0,NaN,NaN,NaN,NaN,NaN,249062.65,4,17,0
4597163,0360884,202006,248056.67,0,31.0,209.0,X,N,0,189901,...,51.0,NaN,NaN,NaN,NaN,NaN,248056.67,4,17,0
4597164,0360884,202007,247047.45,0,32.0,208.0,X,N,0,189901,...,52.0,NaN,NaN,NaN,NaN,NaN,247047.45,4,17,0
4597165,0360884,202008,246034.97,0,33.0,207.0,X,N,0,189901,...,52.0,NaN,NaN,NaN,NaN,NaN,246034.97,4,17,0


In [10]:
df_perf.loc[df_perf['LOAN_SEQUENCE_NUMBER'] == "0207529"]

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,LOAN_AGE,REMAINING_MONTHS_TO_LEGAL_MATURITY,DEFECT_SETTLEMENT_DATE,MODIFICATION_FLAG,ZERO_BALANCE_CODE,ZERO_BALANCE_EFFECTIVE_DATE,...,ELTV,ZERO_BALANCE_REMOVAL_UPB,DELINQUENT_ACCRUED_INTEREST,DELINQUENCY_DUE_TO_DISASTER,BORROWER_ASSISTANCE_STATUS_CODE,CURRENT_MONTH_MODIFICATION_COST,INTEREST_BEARING_UPB,QUARTER,YEAR,REO_ACQUISITION
4400138,0207529,201801,240000.00,0,0.0,360.0,X,N,0,189901,...,79.0,NaN,NaN,NaN,NaN,NaN,240000.00,4,17,0
4400139,0207529,201802,240000.00,0,1.0,359.0,X,N,0,189901,...,79.0,NaN,NaN,NaN,NaN,NaN,240000.00,4,17,0
4400140,0207529,201803,239000.00,0,2.0,358.0,X,N,0,189901,...,78.0,NaN,NaN,NaN,NaN,NaN,239000.00,4,17,0
4400141,0207529,201804,239000.00,0,3.0,357.0,X,N,0,189901,...,77.0,NaN,NaN,NaN,NaN,NaN,239000.00,4,17,0
4400142,0207529,201805,239000.00,0,4.0,356.0,X,N,0,189901,...,72.0,NaN,NaN,NaN,NaN,NaN,239000.00,4,17,0
4400143,0207529,201806,239000.00,0,5.0,355.0,X,N,0,189901,...,71.0,NaN,NaN,NaN,NaN,NaN,239000.00,4,17,0
4400144,0207529,201807,238000.00,0,6.0,354.0,X,N,0,189901,...,74.0,NaN,NaN,NaN,NaN,NaN,238000.00,4,17,0
4400145,0207529,201808,238042.55,1,7.0,353.0,X,N,0,189901,...,74.0,NaN,NaN,NaN,NaN,NaN,238042.55,4,17,0
4400146,0207529,201809,238042.55,2,8.0,352.0,X,N,0,189901,...,74.0,NaN,NaN,NaN,NaN,NaN,238042.55,4,17,0
4400147,0207529,201810,238042.55,3,9.0,351.0,X,N,0,189901,...,74.0,NaN,NaN,NaN,NaN,NaN,238042.55,4,17,0


# 2. Aggregating data like Blumenstock et al. 2020 experiment 4.1

In [11]:
# concatenate df_perf

#df_perf_concatenated = df_perf.groupby('LOAN_SEQUENCE_NUMBER').apply(lambda x: x.iloc[-12:]).reset_index(drop=True)
#df_perf_concatenated.to_csv("SamplePerformanceCombinedConcatenated.csv", header=True, index=False)

In [12]:
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4

df_perf_concatenated = pd.read_csv("SamplePerformanceCombinedConcatenated.csv", dtype=perf_types)
df_perf_concatenated['CURRENT_LOAN_DELINQUENCY_STATUS'] = df_perf_concatenated['CURRENT_LOAN_DELINQUENCY_STATUS'].astype(int)
df_blumenstock = aggregate_to_blumenstock_exp4(df_orig, df_perf_concatenated)

In [13]:
df_blumenstock

,LOAN_SEQUENCE_NUMBER,ORIGINAL_INTEREST_RATE,ORIGINAL_UPB,CREDIT_SCORE,DTI,LTV,BAL_REPAID,T_ACT_12M,T_DEL_30D,T_DEL_60D,DEFAULT,PREPAYMENT
0,0000006,3.750,125000.0,686.0,41.0,65.0,0.000000,12.0,0.0,0.0,0.0,1.0
1,0000027,4.000,44000.0,774.0,30.0,95.0,0.894797,12.0,0.0,0.0,0.0,0.0
2,0000048,3.250,212000.0,808.0,15.0,56.0,0.000000,12.0,0.0,0.0,0.0,1.0
3,0000054,3.875,417000.0,784.0,42.0,66.0,0.000000,12.0,0.0,0.0,0.0,1.0
4,0000076,3.875,40000.0,798.0,22.0,28.0,0.000000,12.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0360865,4.000,451000.0,733.0,39.0,80.0,0.916905,12.0,0.0,0.0,0.0,0.0
99996,0360870,3.750,463000.0,799.0,44.0,60.0,0.000000,11.0,0.0,0.0,0.0,1.0
99997,0360879,3.875,440000.0,812.0,23.0,80.0,0.000000,10.0,0.0,0.0,0.0,1.0
99998,0360883,4.125,74000.0,806.0,20.0,80.0,0.000000,12.0,0.0,0.0,0.0,1.0


In [14]:
df_blumenstock['DEFAULT'].value_counts()

0.0    96598
1.0     3400
Name: DEFAULT, dtype: int64

In [15]:
df_blumenstock['PREPAYMENT'].value_counts()

1.0    60709
0.0    39289
Name: PREPAYMENT, dtype: int64

In [21]:
df_blumenstock['PREPAYMENT']

0        1.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
99995    0.0
99996    1.0
99997    1.0
99998    1.0
99999    0.0
Name: PREPAYMENT, Length: 100000, dtype: float64

In [30]:
row = next(df_blumenstock.iterrows())
print(type(row))

<class 'pandas.core.series.Series'>


In [31]:
# create labels (0 -> prepay, 1 -> default, 2 -> full repay, 3 -> censored)
def label_row(row):

    if row[1]['PREPAYMENT'] == 1:
            return 0
    if row[1]['DEFAULT'] == 1:
            return 1
    else:
        return 3

df_blumenstock['LABEL'] = df_blumenstock.apply(lambda row: label_row(row), axis=1)

TypeError: string indices must be integers

In [16]:
df_blumenstock['LABEL'].value_counts()

KeyError: 'LABEL'

In [17]:
df_blumenstock.count()

LOAN_SEQUENCE_NUMBER      100000
ORIGINAL_INTEREST_RATE    100000
ORIGINAL_UPB              100000
CREDIT_SCORE              100000
DTI                       100000
LTV                       100000
BAL_REPAID                 99998
T_ACT_12M                  99998
T_DEL_30D                  99998
T_DEL_60D                  99998
DEFAULT                    99998
PREPAYMENT                 99998
dtype: int64

In [ ]:
df_perf_concatenated.loc[df_perf_concatenated['CURRENT_LOAN_DELINQUENCY_STATUS'] == 2]

In [ ]:
df_perf_concatenated[(df_perf_concatenated['CURRENT_ACTUAL_UPB'] == 0) & (df_perf_concatenated['ZERO_BALANCE_CODE'] != "01")]

In [ ]:
df_perf_concatenated['REMAINING_MONTHS_TO_LEGAL_MATURITY'].min()

In [ ]:
df_perf.loc[df_perf['REMAINING_MONTHS_TO_LEGAL_MATURITY'] == 2].head(25)

In [ ]:
df_perf.loc[(df_perf['ZERO_BALANCE_CODE'] == "01") & (df_perf['REMAINING_MONTHS_TO_LEGAL_MATURITY'] == 0)].head(25)

In [ ]:
df_perf.loc[df_perf['LOAN_SEQUENCE_NUMBER'] == "0000048"].iloc[-1]

In [ ]:
df_blumenstock.loc[df_blumenstock['LOAN_SEQUENCE_NUMBER'] == "0000307"]

# 3. Expanding Blumenstock et al. 2020 experiment 4.1 to dynamic data